In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('stackoverflow_perguntas.csv')
df.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


In [3]:
len(df)

5408

In [4]:
df.Tags.unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [5]:
df.Tags.nunique()

37

In [6]:
labels = []

for tags in df.Tags.unique():
    for tag in tags.split():
        if not tag in labels:
            labels.append(tag)

labels

['node.js', 'jquery', 'html', 'angular']

In [7]:
for label in labels:
    df[label] = 0
    df.loc[df['Tags'].apply(lambda tags: label in tags), label] = 1

df.head()  

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


In [8]:
df['Labels'] = list(zip(*[df[label] for label in labels]))
df.head()

,Perguntas,Tags,node.js,jquery,html,angular,Labels
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.Perguntas, df.Labels, test_size=0.2, random_state=123)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
tf_idf = TfidfVectorizer(max_features=5000, max_df=0.85)
tf_idf.fit(df.Perguntas)

X_train = tf_idf.transform(X_train)
X_test = tf_idf.transform(X_test)

In [14]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [15]:
regressao_logistica = LogisticRegression()
one_vs_rest = OneVsRestClassifier(estimator=regressao_logistica)

In [16]:
import numpy as np

In [19]:
y_train = np.asarray(list(y_train))
y_test = np.asarray(list(y_test))   

In [20]:
one_vs_rest.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [22]:
one_vs_rest.score(X_test, y_test)

0.4168207024029575

In [24]:
df.Labels.unique()

array([(1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 1),
       (0, 0, 0, 1), (0, 1, 1, 0), (1, 0, 0, 1), (0, 1, 1, 1),
       (0, 1, 0, 1), (1, 0, 1, 0), (1, 1, 0, 0), (1, 0, 1, 1),
       (1, 1, 1, 0)], dtype=object)

In [25]:
1 / df.Labels.nunique()

0.07692307692307693

In [26]:
from sklearn.metrics import hamming_loss 

In [27]:
previsao = one_vs_rest.predict(X_test)
previsao

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       ...,
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0]])

In [29]:
h_loss = hamming_loss(y_test, previsao)
h_loss

0.1883086876155268

In [30]:
df.corr()

C:\Users\Predify\AppData\Local\Temp\ipykernel_13308\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000
